# Comunicación efectiva

## 1. Utilizar la librería `ggplot` o su equivalente `plotnine` para presentar datos de mortalidad en Gran Canaria. Se deben elegir adecuadamente las escalas, presentar gráficos variados y que contrasten la situación en varias regiones de la isla. Bien por municipios o por comarcas, por ejemplo: Norte, Las Palmas, Sur, Centro, etc.

In [1]:
import pandas as pd
import geopandas as gp
from plotnine import *

df = pd.read_csv('../data/defStatTasas.csv').drop(labels = 'Unnamed: 0', axis = 'columns')
display(df)

,Municipio,Año,Sexo,Tipo,Defunciones
0,Agaete,2021,Hombres,I. CIERTAS ENFERMEDADES INFECCIOSAS Y PARASITA...,"0,00"
1,Agüimes,2021,Hombres,I. CIERTAS ENFERMEDADES INFECCIOSAS Y PARASITA...,"24,72"
2,Artenara,2021,Hombres,I. CIERTAS ENFERMEDADES INFECCIOSAS Y PARASITA...,"0,00"
3,Arucas,2021,Hombres,I. CIERTAS ENFERMEDADES INFECCIOSAS Y PARASITA...,"26,17"
4,Firgas,2021,Hombres,I. CIERTAS ENFERMEDADES INFECCIOSAS Y PARASITA...,"26,22"
...,...,...,...,...,...
3187,Telde,2018,Mujeres,XX. CAUSAS EXTERNAS DE MORBILIDAD Y DE MORTALIDAD,"11,51"
3188,Teror,2018,Mujeres,XX. CAUSAS EXTERNAS DE MORBILIDAD Y DE MORTALIDAD,"0,00"
3189,Valsequillo,2018,Mujeres,XX. CAUSAS EXTERNAS DE MORBILIDAD Y DE MORTALIDAD,"0,00"
3190,Valleseco,2018,Mujeres,XX. CAUSAS EXTERNAS DE MORBILIDAD Y DE MORTALIDAD,"111,23"


## 2. Realizar una historia similar por islas